In [1]:
import io
import time
import torch
import torch.nn as nn
import PIL.Image
import io
import numpy as np
import datasets
from torchvision.transforms.v2 import Pad, CenterCrop
from torchvision.transforms.v2.functional import pil_to_tensor, to_pil_image
from types import SimpleNamespace
from autocodec.codec import AutoCodecND, latent_to_pil, pil_to_latent
from piq import LPIPS, DISTS, SSIMLoss

In [2]:
device = "cuda"
lpips_loss = LPIPS().to(device)
dists_loss = DISTS().to(device)
ssim_loss = SSIMLoss().to(device)

/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
inet = datasets.load_dataset("timm/imagenet-1k-wds",split='validation')

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [4]:
# checkpoint = torch.load('../../hf/dance/lsdir_ft_rgb_f16c12.pth', map_location=device,weights_only=False)
# config = checkpoint['config']
# state_dict = checkpoint['state_dict']

# model = AutoCodecND(
#     dim=2,
#     input_channels=config.input_channels,
#     J = int(np.log2(config.F)),
#     latent_dim=config.latent_dim,
#     encoder_depth = config.encoder_depth,
#     encoder_kernel_size = config.encoder_kernel_size,
#     decoder_depth = config.decoder_depth,
#     lightweight_encode = config.lightweight_encode,
#     lightweight_decode = config.lightweight_decode,
# ).to(device)
# model.load_state_dict(state_dict)
# model.eval();

In [5]:
# def compress_and_evaluate(sample):
#     img = sample['jpg'].convert("RGB")
#     aspect = img.width/img.height
#     img = img.resize((int(16*(1024*aspect//16)),1024),resample=PIL.Image.Resampling.LANCZOS)
#     x_orig = pil_to_tensor(img).to(device).unsqueeze(0).to(torch.float) / 127.5 - 1.0
#     orig_dim = x_orig.numel() 

#     t0 = time.time()
#     with torch.no_grad():
#         z = model.encode(x_orig)
#         latent = model.quantize.compand(z).round()
#     webp = latent_to_pil(latent.cpu(), n_bits=8, C=3)
#     buff = io.BytesIO()
#     webp[0].save(buff, format='WEBP', lossless=True)
#     encode_time = time.time() - t0
#     size_bytes = len(buff.getbuffer())
#     t0 = time.time()
#     latent_decoded = pil_to_latent(webp, N=config.latent_dim, n_bits=8, C=3).to(device)
#     with torch.no_grad():
#         x_hat = model.decode(latent_decoded).clamp(-1,1)
#     decode_time = time.time() - t0

#     x_orig_01 = x_orig / 2 + 0.5
#     x_hat_01 = x_hat / 2 + 0.5

#     pixels = img.width * img.height
#     bpp = 8 * size_bytes / pixels
#     mse = torch.nn.functional.mse_loss(x_orig_01[0], x_hat_01[0])
#     PSNR = -10 * mse.log10().item()
#     LPIPS_dB = -10 * np.log10(lpips_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
#     DISTS_dB = -10 * np.log10(dists_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
#     SSIM = 1 - ssim_loss(x_orig_01.to(device), x_hat_01.to(device)).item()

#     return {
#         'pixels': pixels,
#         'bpp': bpp,
#         'PSNR': PSNR,
#         'LPIPS_dB': LPIPS_dB,
#         'DISTS_dB': DISTS_dB,
#         'SSIM': SSIM,
#         'encode_time': encode_time,
#         'decode_time': decode_time,
#     }

# metrics = [
#     'pixels',
#     'bpp',
#     'PSNR',
#     'LPIPS_dB',
#     'DISTS_dB',
#     'SSIM',
#     'encode_time',
#     'decode_time',
# ]

In [6]:
def compress_and_evaluate_webp(sample, q):
    img = sample['jpg'].convert("RGB")
    aspect = img.width/img.height
    img = img.resize((int(16*(1024*aspect//16)),1024),resample=PIL.Image.Resampling.LANCZOS)
    x_orig = pil_to_tensor(img).to(device).unsqueeze(0).to(torch.float) / 127.5 - 1.0
    orig_dim = x_orig.numel() 

    t0 = time.time()
    buff = io.BytesIO()
    img.save(buff, format='WEBP', quality=q)
    encode_time = time.time() - t0
    size_bytes = len(buff.getbuffer())
    t0 = time.time()
    webp = PIL.Image.open(buff)
    decode_time = time.time() - t0
    x_hat = pil_to_tensor(webp).to(device).unsqueeze(0).to(torch.float) / 127.5 - 1.0

    x_orig_01 = x_orig / 2 + 0.5
    x_hat_01 = x_hat / 2 + 0.5

    pixels = img.width * img.height
    bpp = 8 * size_bytes / pixels
    mse = torch.nn.functional.mse_loss(x_orig_01[0], x_hat_01[0])
    PSNR = -10 * mse.log10().item()
    LPIPS_dB = -10 * np.log10(lpips_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
    DISTS_dB = -10 * np.log10(dists_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
    SSIM = 1 - ssim_loss(x_orig_01.to(device), x_hat_01.to(device)).item()

    return {
        'pixels': pixels,
        'bpp': bpp,
        'PSNR': PSNR,
        'LPIPS_dB': LPIPS_dB,
        'DISTS_dB': DISTS_dB,
        'SSIM': SSIM,
        'encode_time': encode_time,
        'decode_time': decode_time,
    }

metrics = [
    'pixels',
    'bpp',
    'PSNR',
    'LPIPS_dB',
    'DISTS_dB',
    'SSIM',
    'encode_time',
    'decode_time',
]

webp q=0

In [10]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,0))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.09295122017843688
PSNR: 28.334981180348638
LPIPS_dB: 3.782436879724428
DISTS_dB: 8.385171812353674
SSIM: 0.8658390640581058
encode_time: 0.03966331481933594
decode_time: 0.0003862267328520953
31.6725413535132 MP/sec


webp q=5

In [8]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,5))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.16808051894428486
PSNR: 30.94224475197873
LPIPS_dB: 4.916882737095818
DISTS_dB: 10.549339892410623
SSIM: 0.9287441000973774
encode_time: 0.044705461394988884
decode_time: 0.0003886356697244159
28.205249284297587 MP/sec


webp q=10

In [7]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,10))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.2012685052924489
PSNR: 31.77052897164377
LPIPS_dB: 5.247660671771126
DISTS_dB: 11.179777161462045
SSIM: 0.9404032268261505
encode_time: 0.04622692198066388
decode_time: 0.00030636888439372433
27.28831767009428 MP/sec


webp q=20

In [9]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,20))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.25711650084926757
PSNR: 32.92799842307123
LPIPS_dB: 5.752427074729006
DISTS_dB: 12.105877406208801
SSIM: 0.9540889172862141
encode_time: 0.048235851576772786
decode_time: 0.00039599911641266387
26.191032012979594 MP/sec


webp q=50

In [11]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,50))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.4230773113302287
PSNR: 35.44099694591458
LPIPS_dB: 6.939734946827561
DISTS_dB: 14.281323711338276
SSIM: 0.9728789905370292
encode_time: 0.05272707696688377
decode_time: 0.0003973344119928651
24.003459940771414 MP/sec


webp q=75

In [12]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,75))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.5793115656704492
PSNR: 37.07799157853854
LPIPS_dB: 7.963087386342984
DISTS_dB: 16.063846417206793
SSIM: 0.9813222195006023
encode_time: 0.0560637886746455
decode_time: 0.00039520809205911925
22.55939165295544 MP/sec


webp q=85

In [13]:
results_dataset = inet.select(range(0,50000,53)).map(lambda s: compress_and_evaluate_webp(s,85))

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

mean
---
pixels: 1247960.9491525423
bpp: 0.8900880425602385
PSNR: 39.481595423767125
LPIPS_dB: 9.857538251463039
DISTS_dB: 19.39745151739412
SSIM: 0.989707756724398
encode_time: 0.06217083764278283
decode_time: 0.0004185877614102121
20.34589394907377 MP/sec


f16c12 lsdir+inet+lsdir

In [7]:
results_dataset = inet.map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

mean
---
pixels: 1233140.44928
bpp: 0.14419030833425764
PSNR: 31.37415174732208
LPIPS_dB: 5.915171539826647
DISTS_dB: 11.889201100445513
SSIM: 0.9438236615478992
encode_time: 0.01610152611732483
decode_time: 0.008204229717254638
79.54539609851899 MP/sec


In [14]:
24/0.14419030833425764

166.44669310480924

f8c12 lsdir

In [ ]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

f8c12 lsdir+inet

In [7]:
results_dataset = inet.map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

mean
---
pixels: 1233140.44928
bpp: 0.6455606527768093
PSNR: 38.895700521564486
LPIPS_dB: 10.482115628613725
DISTS_dB: 19.412052084736796
SSIM: 0.9871415659821033
encode_time: 0.058272379055023193
decode_time: 0.007950794057846069
23.599175936410298 MP/sec


f8c12 lsdir+inet+lsdir

In [ ]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

f16c48 lsdir+inet

In [7]:
results_dataset = inet.map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

mean
---
pixels: 1233140.44928
bpp: 0.6313282907855455
PSNR: 38.88292310328484
LPIPS_dB: 10.749157017407448
DISTS_dB: 20.078580878680228
SSIM: 0.9875113545906544
encode_time: 0.033106667222976685
decode_time: 0.008710551104545593
38.27494239613218 MP/sec
